# Animal Detection Using YOLOv3 & YOLOv4
Both YOLOv3 and YOLOv4 use the Darknet open source neural network framework written in C and CUDA. 

This framework supports both CPU and GPU computation.

In [1]:
!apt update
!uname -m && cat /etc/*release
!gcc --version
!uname -r

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:13 http://security.ubu

In [2]:
!nvidia-smi

Sun Feb 27 11:17:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

To access the Google Drive folder and files

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the modified version of the Darknet framework

In [4]:
%rm -r darknet
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet/

rm: cannot remove 'darknet': No such file or directory
Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.03 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (10346/10346), done.
/content/darknet


## Training YOLOv3 and YOLOv4 on a custom dataset
1.   Compile Darknet with `GPU=1 CUDNN=1 CUDNN_HALF=1 OPENCV=1` in the Makefile


In [5]:
%pycat Makefile

**Changing** the variables to include GPU compilation using OpenCV.

In [6]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

%pycat Makefile

In [7]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
  

2.   Create a new configuration file with the same content as in `yolov3.cfg` or `yolov4-custom.cfg` depending on if you want to train on YOLOv3 or YOLOv4.

**Change** the following in the configuration file:
*   change line batch to 64.
*   change line subdivisions to 16.
*   change line max_batches to 2000 x classes and not less than 6000 iterations.
*   change line steps to 80% and 90% of max_batches.
*   change line width and height to 416 or any value multiple of 32.
*   change line classes to your number of objects in each of the 3 yolo layers.
*   change filters to 3 x (classes + 5) in the convolutional layer before each of the 3 yolo layer.


In [8]:
%cp -r "../drive/My Drive/Colab Notebooks/Darknet/data/yolov3-obj.cfg" cfg/
%cp -r "../drive/My Drive/Colab Notebooks/Darknet/data/yolov4-obj.cfg" cfg/

In [9]:
%pycat cfg/yolov3-obj.cfg

In [10]:
%pycat cfg/yolov4-obj.cfg

3.   Create file obj.names in the directory `*build\darknet\x64\data\*` with objects names - each in new line

In [ ]:
classes = """Eagle
Elephant
Lion
Monkey
Parrot
"""

file = """text_file = open("build/darknet/x64/data/obj.names", "w");text_file.write(classes);text_file.close()""" 

exec(file)
%pycat build/darknet/x64/data/obj.names

4.   Create file `obj.data` in the directory `build\darknet\x64\data\` containing (where `classes = number of objects`):


In [12]:
obj_data = """classes= 5
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/valid.txt
names = build/darknet/x64/data/obj.names
backup = build/darknet/x64/backup/
"""

file = """text_file = open("build/darknet/x64/data/obj.data", "w");text_file.write(obj_data);text_file.close()""" 

exec(file)
%pycat build/darknet/x64/data/obj.data

5.   Put image and text files (`.jpg` and `.txt`) of your objects in the directory `build/darknet/x64/data/obj/`



In [13]:
%ls '../drive/My Drive/Colab Notebooks/Darknet/'

backup/  data/


In [14]:
%mkdir build/darknet/x64/data/obj
%cp -r "../drive/My Drive/Colab Notebooks/Darknet/data/obj/train/." build/darknet/x64/data/obj/
%cp -r "../drive/My Drive/Colab Notebooks/Darknet/data/obj/valid/." build/darknet/x64/data/obj/

In [15]:
#total number of files
%ls -1 build/darknet/x64/data/obj/ | wc -l

18820


In [16]:
##total number of .jpg & .txt files
%ls -1 build/darknet/x64/data/obj/*.jpg | wc -l
%ls -1 build/darknet/x64/data/obj/*.txt | wc -l

9410
9410


6.   Create file `train.txt` and  `valid.txt` in directory `build\darknet\x64\data\`, with filenames of your images, each filename in new line, with path relative to `./darknet`, for example containing:

In [17]:
%cp -r "/content/drive/My Drive/Colab Notebooks/Darknet/data/train.txt" build/darknet/x64/data/
%cp -r "/content/drive/My Drive/Colab Notebooks/Darknet/data/valid.txt" build/darknet/x64/data/

In [18]:
#Count number of files 
!wc -l build/darknet/x64/data/train.txt
!wc -l build/darknet/x64/data/valid.txt

7956 build/darknet/x64/data/train.txt
1454 build/darknet/x64/data/valid.txt


In [19]:
%pycat build/darknet/x64/data/valid.txt

7.  Download pre-trained weights for the convolutional layers and put to the directory `build\darknet\x64`
*   For `yolov4.cfg`, `yolov4-custom.cfg` (162 MB): [yolov4.conv.137](https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137)
*   For `yolov3.cfg`, `yolov3-spp.cfg` (154 MB): [darknet53.conv.74](https://pjreddie.com/media/files/darknet53.conv.74)


In [ ]:
%cp '../drive/My Drive/Colab Notebooks/Darknet/data/yolov4.conv.137' build/darknet/x64/
%cp '../drive/My Drive/Colab Notebooks/Darknet/data/darknet53.conv.74' build/darknet/x64/

8.  Start training by using the command line: `./darknet detector train data/obj.data yolo-obj.cfg yolov4.conv.137` 
*   File `yolo-obj_last.weights` will be saved to the `build\darknet\x64\backup` every 100 iterations
*   File `yolo-obj_xxxx.weights` will be saved to the `build\darknet\x64\backup` every 1000 iterations
*   To disable the Loss-Window use the flag `-dont_show` in the end of the command when you train on your computer
*   To see the mAP & Loss-chart during training on remote server without GUI, use the following flag `-dont_show -mjpeg_port 8090 -map` and then open the URL `https://localhost:8090` in a Firefox browser

For training with mAP calculation every 4 epochs (set `valid=valid.txt` or `valid=train.txt` in the obj.data file) and run the following command:
```
./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map
```

In [ ]:
%cat build/darknet/x64/data/obj.data

classes= 5
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/valid.txt
names = build/darknet/x64/data/obj.names
backup = build/darknet/x64/backup/


In [ ]:
# To train YOLOv4 on Linux use command:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolov4-obj.cfg build/darknet/x64/yolov4.conv.137 -dont_show -map | tee backup/logs.txt

In [ ]:
# To train on Linux use command:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolov3-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map | tee backup/logs.txt

In [20]:
%cp '../drive/My Drive/Colab Notebooks/Darknet/backup/yolov3-obj_800.weights' build/darknet/x64/

In [ ]:
# To train on Linux use command:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolov3-obj.cfg build/darknet/x64/yolov3-obj_800.weights -dont_show -map | tee backup/logs.txt

Streaming output truncated to the last 5000 lines.
 total_bbox = 96092, rewritten_bbox = 0.282021 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.765277), count: 3, class_loss = 0.496262, iou_loss = 0.164266, total_loss = 0.660528 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.712942), count: 3, class_loss = 1.053861, iou_loss = 0.308352, total_loss = 1.362213 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000339, iou_loss = 0.000000, total_loss = 0.000339 
 total_bbox = 96098, rewritten_bbox = 0.282004 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.822712), count: 4, class_loss = 0.988621, iou_loss = 0.232955, total_loss = 1.221576 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.004592, iou_loss = 0.000000, total_loss = 0.004592 
v3 (mse 

In [ ]:
%ls build/darknet/x64/backup/*.weights

In [ ]:
# copy pre-trained weights files to google drive
%cp -r build/darknet/x64/backup/*.weights "/content/drive/My Drive/Colab Notebooks/Darknet/backup/"

In [ ]:
%ls '../drive/My Drive/Colab Notebooks/Darknet/backup/'

9.  After training is complete - get result `yolo-obj_final.weights` from path `build\darknet\x64\backup\` and then run the command:

In [ ]:
!./darknet detector test data/obj.data yolov4-obj.cfg yolo-obj_final.weights

In [ ]:
!./darknet detector test data/obj.data yolov3-obj.cfg yolo-obj_final.weights